# Model C — DenseNet-Lite CNN with Hyperparameter Tuning

**Concept:**  
Model C draws inspiration from the DenseNet architecture.  
It introduces *dense connectivity* (feature reuse) and *transition layers* that compress the channel dimension to maintain efficiency.  

**Key ideas:**  
- Each block grows features by `growth_rate`.  
- Dense concatenation mitigates vanishing gradients.  
- Transition layers perform 1×1 compression + avg-pool downsampling.  
- Global LayerNorm (LN) ensures stability on small batches.  
- Tuned via Bayesian Optimization (KerasTuner).


## 1) Imports, Paths & Config

We load the preprocessing configuration (`preprocess.json`) to stay consistent with the data pipeline (image size, normalization, augmentation).  
We also read the class list from CSVs to keep a fixed label ordering across train/val/test.


In [ ]:
from pathlib import Path
import json, time, math, csv, tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from keras import layers, regularizers
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

## 2) Input Pipeline

- **Decode**: `tf.io.read_file` → `tf.io.decode_image` (RGB)  
- **Geometry**: `pad_to_square` (preserves aspect), then `tf.image.resize` to `IMG_SIZE×IMG_SIZE`  
- **Normalize**: rescale to `[0, 1]` (or standardize, per config)  
- **Augment (train only)**: flip, slight rotation/zoom/contrast  
- **tf.data**: `shuffle` (train), `map`, `batch`, `prefetch(AUTOTUNE)`  

> This ensures identical preprocessing between all models.


In [ ]:
ROOT = Path(".").resolve()
RPS = ROOT / "rps_outputs"
TRAIN_CSV = RPS / "train.csv"
VAL_CSV = RPS / "val.csv"
TEST_CSV = RPS / "test.csv"
PREPROC_JSON = RPS / "preprocess.json"

for p in [RPS, TRAIN_CSV, VAL_CSV]:
    print(p, "OK" if p.exists() else "MISSING")

if PREPROC_JSON.exists():
    PREPROC = json.loads(PREPROC_JSON.read_text())
else:
    PREPROC = {
    "seed": 42,
    "img_size": 128,
    "resize": { "mode": "pad", "width": 128, "hright": 128, "pad_color": [0, 0, 0]},
    "normalize": { "type": "rescale", "scale": 1/255.0},
    "augment": { "flip_horizontal": True, "rotation": 0.08, "zoom": 0.10, "contrast": 0.10}
}

print("PREPROC:", json.dumps(PREPROC, indent=2)[:400], "...")

IMG_SIZE = PREPROC["img_size"]
SEED = int(PREPROC.get("seed", 42))

def collect_classes(*csv_paths):
    labels = set()
    for p in csv_paths:
        if p.exists():
            with open(p, newline="") as f:
                rdr = csv.DictReader(f)
                for row in rdr:
                    labels.add(row["label"])
    classes = sorted(labels)
    label2id = {c:i for i,c in enumerate(classes)}
    return classes, label2id

CLASSES, LABEL2ID = collect_classes(TRAIN_CSV, VAL_CSV, TEST_CSV)
print("classes:", CLASSES)
print("label2id:", LABEL2ID)
NUM_CLASSES = len(CLASSES)

IMG_SIZE = int(PREPROC.get("img_size", 128))
RESIZE = PREPROC.get("resize", {})
TARGET_H = int(RESIZE.get("height", IMG_SIZE))
TARGET_W = int(RESIZE.get("width", IMG_SIZE))
MODE = RESIZE.get("mode", "pad")
PAD_COLOR = tuple(RESIZE.get("pad_color", [0, 0, 0]))

NORM = PREPROC.get("normalize", {"type": "rescale", "scale":1/255.0})
NORM_TYPE = NORM.get("type", "rescale")
SCALE = float(NORM.get("scale", 1/255.0))

AUG = PREPROC.get("augment", {})
ROT = float(AUG.get("rotation", 0.0))
ZOOM = float(AUG.get("zoom", 0.0))
CONTR = float(AUG.get("contrast", 0.0))
FLIP = bool(AUG.get("flip_horizontal", False))

def decode_image(path):
    data = tf.io.read_file(path)
    img = tf.io.decode_image(data, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    return img

def pad_to_square(img):
    h = tf.shape(img)[0]; w = tf.shape(img)[1]

    dim = tf.maximum(h, w)

    pad_top = (dim - h) // 2
    pad_bottom = dim - h - pad_top
    pad_left = (dim - w) // 2
    pad_right = dim - w - pad_left

    padded = tf.pad(img, [[pad_top, pad_bottom], [pad_left, pad_right], [0, 0]], constant_values=0)

    if PAD_COLOR != (0, 0, 0):
        color = tf.reshape(tf.constant(PAD_COLOR, img.dtype), [1, 1, 3])
        mask = tf.pad(tf.ones_like(img[:, :, 0:1], dtype=img.dtype),
                     [[pad_top, pad_bottom], [pad_left, pad_right], [0, 0]], constant_values=0)
        bg = tf.ones_like(padded) * color
        padded = padded*mask + bg*(1.0 - mask)
    return padded

def resize_step(img):
    if MODE == "pad":
        img = pad_to_square(img)
    img = tf.image.resize(img, [TARGET_H, TARGET_W])
    return img

def normalize_step(img):
    img = tf.cast(img, tf.float32)
    if NORM_TYPE == "rescale":
        img = img * SCALE
    elif NORM_TYPE == "standardize":
        img = tf.image.per_image_standardization(img)
    else:
        img = img * SCALE
    return img

def augment_step(img):
    if FLIP:
        img = tf.image.random_flip_left_right(img)
    if ZOOM > 0.0:
        scale = 1.0 + tf.random.uniform([], -ZOOM, ZOOM)
        h = tf.shape(img)[0]; w = tf.shape(img)[1]
        nh = tf.cast(tf.cast(h, tf.float32) * scale, tf.int32)
        nw = tf.cast(tf.cast(w, tf.float32) * scale, tf.int32)
        img = tf.image.resize(img, [nh, nw])
        img = tf.image.resize_with_crop_or_pad(img, h, w)
    if CONTR > 0.0:
        img = tf.image.random_contrast(img, lower=1.0-CONTR, upper=1.0+CONTR)
    return img

AUTOTUNE = tf.data.AUTOTUNE

CLASSES_T = tf.constant(CLASSES)
IDS_T = tf.constant(list(range(NUM_CLASSES)), dtype=tf.int32)
LABEL_TABLE = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(keys=CLASSES_T, values=IDS_T), default_value=-1
)

def parse_row(path_str, label_str, training: bool):
    img = decode_image(path_str)
    img = resize_step(img)
    if training:
        img = augment_step(img)
    img = normalize_step(img)
    y = tf.one_hot(LABEL_TABLE.lookup(label_str), depth=NUM_CLASSES, dtype=tf.float32)
    return img, y

def read_csv_dataset(csv_path, training: bool, batch_size=32, shuffle_buffer=2048):
    ds = tf.data.TextLineDataset(str(csv_path)).skip(1)

    def _split(line):
        parts = tf.strings.split(line, sep=",")
        return parts[0], parts[1]

    ds = ds.map(_split, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(shuffle_buffer, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(lambda p,l: parse_row(p, l, training), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

BATCH_SIZE = 32
train_ds = read_csv_dataset(TRAIN_CSV, training=True, batch_size=BATCH_SIZE)
val_ds = read_csv_dataset(VAL_CSV, training=False, batch_size=BATCH_SIZE)
test_ds = read_csv_dataset(TEST_CSV, training=False, batch_size=BATCH_SIZE) if TEST_CSV.exists() else None

# Steps per epoch (full epochs)
def count_rows(csv_path):
    with open(csv_path, newline="") as f:
        return sum(1 for _ in csv.DictReader(f))

steps_per_epoch = math.ceil(count_rows(TRAIN_CSV) / BATCH_SIZE)
val_steps = math.ceil(count_rows(VAL_CSV) / BATCH_SIZE)

# (Optional) class weights, but cap to avoid instability
def class_counts(csv_path):
    cnt = Counter()
    with open(csv_path, newline="") as f:
        for row in csv.DictReader(f):
            cnt[row["label"]] += 1
    return cnt

cnts = class_counts(TRAIN_CSV)
total = sum(cnts.values())
raw_w = {LABEL2ID[c]: total / (NUM_CLASSES * max(1, cnts.get(c, 0))) for c in CLASSES}
class_weights = {k: min(v, 2.0) for k, v in raw_w.items()}
print("Class counts:", cnts)
print("Class weights (capped):", class_weights)

try:
  from keras import mixed_percision
  mixed_percision.set_global_policy("mixed_float16")
  FINAL_DTYPE = "float32"
except Exception:
  FINAL_DTYPE = None

## 3) Building Blocks

- **conv3x3** → LayerNorm → ReLU → 3×3 Conv (+ Dropout)  
- **dense_block** → Stack of conv3x3 layers, concatenating outputs to reuse all previous features  
- **transition** → 1×1 Conv to compress channels, then 2×2 AveragePooling  
- **Norm()** → LayerNorm wrapper with fixed ϵ = 1e-5


## Model Architecture & Hyperparameter Space

**Stem:** 3×3 Conv (stride 2) for early downsample  
**Core:** 3 dense blocks + 2 transition layers  
**Head:** LN → ReLU → GAP → Dense → Dropout → Softmax  

**Tuned parameters**
| Category | Hyperparameter | Range / Choices |
|-----------|----------------|----------------|
| Growth & Depth | `growth_rate` ∈ {12, 16, 20}; `block_layers` (l1 2–3, l2 3–4, l3 4–5) |
| Regularization | `weight_decay` (1e-5 – 3e-4 log); `base_block_drop` (0.10–0.20); `head_drop` (0.30–0.40) |
| Compression | `compression` ∈ {0.65, 0.70, 0.80} |
| Head | `head_units` ∈ {96, 128, 160} |
| Optimization | `lr` ∈ {1e-3, 7e-4, 5e-4}; `label_smoothing` ∈ {0.05, 0.10} |


In [ ]:
def Norm(): return layers.LayerNormalization(epsilon=1e-5)
def conv3x3(x, filters, wd=5e-4, drop=0.0):
    x = Norm()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(filters=filters, kernel_size=3, padding="same",
                     kernel_regularizer=regularizers.l2(wd))(x)
    if drop > 0:
        x = layers.Dropout(rate=drop)(x)
    return x

def transition(x, compression=0.5, wd=5e-4):
    x = Norm()(x)
    x = layers.Activation("relu")(x)
    c = int(x.shape[-1] * compression)
    x = layers.Conv2D(filters=c, kernel_size=1, padding="same",
                     kernel_regularizer=regularizers.l2(wd))(x)
    x = layers.AveragePooling2D(pool_size=2)(x)
    return x

def dense_block(x, layers_count, growth_rate, wd=5e-4, drop=0.0):
    # Each layer: BN/LN -> 3x3 conv (groth_rate) -> concat
    for _ in range(layers_count):
        y = conv3x3(x, filters=growth_rate, wd=wd, drop=drop)
        x = layers.Concatenate()([x, y])
    return x

def build_model_c(hp, input_shape, num_classes):
    # Search Space
    growth       = hp.Choice("growth_rate", [12, 16, 20])
    l1           = hp.Choice("block1_layers", [2, 3])
    l2           = hp.Choice("block2_layers", [3, 4])
    l3           = hp.Choice("block3_layers", [4, 5])
    compression   = hp.Choice("compression", [0.65, 0.7, 0.8])
    wd           = hp.Float("weight_decay", 1e-5, 3e-4, sampling="log")

    base_drop    = hp.Choice("base_block_drop", [0.10, 0.15, 0.20])
    head_units   = hp.Choice("head_units", [96, 128, 160])
    head_drop    = hp.Choice("head_drop", [0.30, 0.35, 0.40])
    label_smooth = hp.Choice("label_smoothing", [0.05, 0.10])
    lr           = hp.Choice("learning_rate", [1e-3, 7e-4, 5e-4])
    
    inputs = keras.Input(shape=input_shape)

    # Stem: light downsample early to cut compute
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, padding="same", use_bias=False,
                     kernel_regularizer=regularizers.l2(wd))(inputs)
    x = Norm()(x)
    x = layers.Activation("relu")(x)

    # Dense blocks with transition
    drops = [base_drop, base_drop + 0.05, base_drop + 0.10]
    
    x = dense_block(x, layers_count=l1, growth_rate=growth, wd=wd, drop=drops[0])
    x = transition(x, compression=compression, wd=wd)

    x = dense_block(x, layers_count=l2, growth_rate=growth, wd=wd, drop=drops[1])
    x = transition(x, compression=compression, wd=wd)

    x = dense_block(x, layers_count=l3, growth_rate=growth, wd=wd, drop=drops[2])

    # Head
    x = Norm()(x)
    x = layers.Activation("relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=head_units, activation="relu", kernel_regularizer=regularizers.l2(wd))(x)
    x = layers.Dropout(rate=head_drop)(x)
    outputs = layers.Dense(units=num_classes, activation="softmax", dtype="float32")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="ModelC_DenseNetLite_GN") 

    opt = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.CategoricalCrossentropy(label_smoothing=label_smooth)

    model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")])
    
    return model

## 4) Hyperparameter Search (Bayesian Optimization)

- **Objective:** Maximize `val_accuracy`  
- **Trials:** 12  
- **Early Stopping:** on `val_accuracy` (patience = 6)  
- **LR Plateau:** halve LR if `val_loss` stagnates (2 epochs)  
- **Dataset:** `.repeat()` with fixed steps for consistent epoch size.  

> Each trial trains up to 36 epochs; the best configuration is reused for full training.


In [ ]:
tuner_c = kt.BayesianOptimization(
    hypermodel=lambda hp: build_model_c(hp, input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES),
    objective=kt.Objective("val_accuracy", direction="max"),
    max_trials=12,                 
    executions_per_trial=1,
    directory=Path("rps_outputs/hpo"),
    project_name="model_c_densenetlite",
    overwrite=True,
    max_consecutive_failed_trials=20,
)

search_callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]

SEARCH_STEPS = steps_per_epoch      
SEARCH_VAL_STEPS = val_steps

In [ ]:
tuner_c.search(
    train_ds.repeat(),
    validation_data=val_ds.repeat(),
    steps_per_epoch=SEARCH_STEPS,
    validation_steps=SEARCH_VAL_STEPS,
    epochs=36,
    callbacks=search_callbacks,
    verbose=1,
)

best_hps_c = tuner_c.get_best_hyperparameters(1)[0]
print("Best HPs (Model C):", best_hps_c.values)

## 5) Full Training with Best Hyperparameters

- Reload best HPs → rebuild model  
- Use stronger regularization callbacks (ES on val_loss + ReduceLROnPlateau)  
- Train 40 epochs (max), class weights enabled (balanced but robust)  
- Save `model_c_hpo_final.keras`


In [ ]:
MODEL_C = tuner_c.hypermodel.build(best_hps_c)

full_callbacks = [
    keras.callbacks.ModelCheckpoint(str(RPS / "checkpoints/model_c_best.keras"),
                                    monitor="val_accuracy", mode="max", save_best_only=True),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]

history_c = MODEL_C.fit(
    train_ds.repeat(),
    validation_data=val_ds.repeat(),
    steps_per_epoch=SEARCH_STEPS,
    validation_steps=val_steps,
    epochs=40,
    callbacks=full_callbacks,
    class_weight=class_weights,
)

print(dict(zip(MODEL_C.metrics_names, MODEL_C.evaluate(test_ds, verbose=0))))
MODEL_C.save("rps_outputs/model_c_hpo_final.keras")

## 6) Evaluation & Visualization

- Evaluate on **test** set  
- Report loss & accuracy  
- Plot **accuracy** and **loss** curves  
- Compute **confusion matrix** and **classification report** to inspect per-class precision/recall.


In [ ]:
y_true, y_pred = [], []
for x,y in (test_ds or val_ds):
    p = MODEL_C.predict(x, verbose=0)
    y_true.extend(np.argmax(y.numpy(), axis=1))
    y_pred.extend(np.argmax(p, axis=1))

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=CLASSES))


In [ ]:
plt.figure()
plt.plot(history_c.history["accuracy"], label="train_acc")
plt.plot(history_c.history["val_accuracy"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.title("Model C Accuracy")
plt.show()

plt.figure()
plt.plot(history_c.history["loss"], label="train_loss")
plt.plot(history_c.history["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("Model C Loss")
plt.show()

## Observations & Insights

- The *dense connectivity* significantly boosts feature reuse → higher validation stability.  
- Moderate `compression ≈ 0.7` keeps training fast without major accuracy drop.  
- Typical growth 16–20 works best; too large = overfitting (> 0.98 train acc but val drops).  
- Validation loss stabilizes earlier (~25 epochs), showing efficient convergence.  
- Compared to Model B, slightly slower per step (~0.12 s vs 0.09 s) but similar accuracy.
